# Vector Stores

- SQL/NoSQL/Graph/Vector databases
- provides native support for storing and performing queries on vector data

## Example databases
- PostgreSQL (with `pg_vector` plugin)
- Apache Cassandra
- Elasticsearch / OpenSearch (AWS)
- Facebook AI Similarity Search (`FAISS`)
- Pinecone
- AWS Kendra

![Vector stores databases](./images/7-vector-stores-databases.png)

## Data types
![Vector stores data](./images/7-vector-stores-data.png)

![Vector stores embeddings](./images/7-vector-stores-embeddings.png)

- vector databases use special search techniques known as Approximate Nearest Neighbor (ANN) search, K-Nearest Neighbour (KNN), Cosine similarity, etc

## Import dependencies

In [ ]:
var BedrockEmbeddings = require('@langchain/community/embeddings/bedrock').BedrockEmbeddings;
var Bedrock = require('@langchain/community/llms/bedrock').Bedrock;

## Instantiate the `model` client

In [ ]:
var model = new Bedrock({
    model_id:'amazon.titan-text-express-v1',
    temperature: 1,
    maxTokenCount: 512,
    topP: 0.9,
    verbose: true
});

## Instantiate the `embeddings model` client

In [ ]:
var embeddingsClient = new BedrockEmbeddings({
    model:'amazon.titan-embed-text-v2:0',
    region:'us-east-1'
});

## Load data into memory

In [ ]:
var PDFLoader = require('@langchain/community/document_loaders/fs/pdf').PDFLoader;

var pdfLoaderClient = new PDFLoader("/workspace/packages/llm/src/notebooks/data/7-2022-Shareholder-Letter.pdf");

var rawPdfDocuments;
pdfLoaderClient.load().then((pdfDocuments) => {
    rawPdfDocuments = pdfDocuments;
    console.log(pdfDocuments);
});

## Split documents

In [ ]:
var RecursiveCharacterTextSplitter = require("langchain/text_splitter").RecursiveCharacterTextSplitter;

var splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 300,
    chunkOverlap: 30
});

In [ ]:
var splitDocs;

splitter.splitDocuments(rawPdfDocuments).then((data) => {
    splitDocs = data;
    console.log(splitDocs);
});

## Create vector store using `documents` and `embeddings model`

In [ ]:
var MemoryVectorStore = require('langchain/vectorstores/memory').MemoryVectorStore;

var vectorStore;

MemoryVectorStore.fromDocuments(splitDocs, embeddingsClient).then((store) => {
    vectorStore = store;

});

## Generate a retriever function

- a function to be provided as a tool to `Langchain` that gets data from vector store to be used by model

In [ ]:
var retriever = vectorStore.asRetriever({k: 3});

## Create prompt

In [ ]:
var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;
var prompt = ChatPromptTemplate.fromTemplate(`
    Answer the user question.
    Context: {context}
    Question: {input}
`);

## Create basic documents chain

In [ ]:
var createStuffDocumentsChain = require("langchain/chains/combine_documents").createStuffDocumentsChain;
var combineDocsChain;

createStuffDocumentsChain({
    llm: model,
    prompt,
}).then((chain) => combineDocsChain = chain);

## Create retrieval chain

- chain that is enhanced compared to previous ones, uses the vector store retriever

In [ ]:
var createRetrievalChain = require('langchain/chains/retrieval').createRetrievalChain;

var retrievalChain;

createRetrievalChain({
    combineDocsChain,
    retriever
}).then((chain) => retrievalChain = chain);

## Configure input query

In [ ]:
var input = "Who is Andy Jassy?";

## Invoke LLM

- we invoke LLM with only relevant information from vector store based on similarity search done on embeddings of all input pdf chunks and embedding of input (handled by chain)

In [ ]:
retrievalChain.invoke({
    input,
}).then((response) => console.log(response));

![High level flow](./images/6-embeddings-high-level.png)

## Pinecone

- due to time savings reasons (no need to deploy VPC/RDS/OpenSearch clusters/setup extra local containers) we will be using `Pinecone` `SaaS` vector store
- website is [https://www.pinecone.io/](https://www.pinecone.io/)
- there is a free tier available for one index
- we will get API key for Pinecone and leverage that from local/AWS.

### Setup
1. We need to also install `@langchain/pinecone` by doing

```shell
yarn add @langchain/pinecone
```
2. Create Pinecone account []()
3. Create index `codechat2024`
Use parameters from [AWS Bedrock](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-setup.html) for index setup.

## Import environment variables for API keys

In [5]:
require('dotenv').config({path: require('path').resolve(__dirname, '../../.env')});

{
  parsed: {
    PINECONE_API_KEY: 'da6a78d1-66bd-477a-bebf-e790e968411d',
    PINECONE_INDEX: 'chatbot'
  }
}

## Initialize Pinecone Vector Store

In [6]:
var Pinecone = require('@pinecone-database/pinecone').Pinecone;
var PineconeStore = require('@langchain/pinecone').PineconeStore;
var BedrockEmbeddings = require('@langchain/community/embeddings/bedrock').BedrockEmbeddings;

var embeddingsClient = new BedrockEmbeddings({
    model:'amazon.titan-embed-text-v2:0',
    region:'us-east-1'
});

var pineconeClient = new Pinecone({
    apiKey: process.env.PINECONE_API_KEY,
});

var pineconeIndex = pineconeClient.index(process.env.PINECONE_INDEX);
var pineconeStore = new PineconeStore(embeddingsClient, {
    pineconeIndex
});

## Get `Langchain` `Documents` from existing `PDF`

In [7]:
var PDFLoader = require('@langchain/community/document_loaders/fs/pdf').PDFLoader;

var pdfLoaderClient = new PDFLoader("/workspace/packages/llm/src/notebooks/data/7-2022-Shareholder-Letter.pdf");

var rawPdfDocuments;
pdfLoaderClient.load().then((pdfDocuments) => {
    rawPdfDocuments = pdfDocuments;
    console.log(pdfDocuments);
});

Promise { <pending> }

[
  Document {
    pageContent: 'Dear shareholders:\n' +
      'As I sit down to write my second annual shareholder letter as CEO, I find myself optimistic and energized\n' +
      'by what lies ahead for Amazon. Despite 2022 being one of the harder macroeconomic years in recent memory,\n' +
      'and with some of our own operating challenges to boot, we still found a way to grow demand (on top of\n' +
      'the unprecedented growth we experienced in the first half of the pandemic). We innovated in our largest\n' +
      'businesses to meaningfully improve customer experience short and long term. And, we made important\n' +
      'adjustments in our investment decisions and the way in which we’ll invent moving forward, while still\n' +
      'preserving the long-term investments that we believe can change the future of Amazon for customers,\n' +
      'shareholders, and employees.\n' +
      'While there were an unusual number of simultaneous challenges this past year, the reality is

In [ ]:
var RecursiveCharacterTextSplitter = require("langchain/text_splitter").RecursiveCharacterTextSplitter;

var splitter = new RecursiveCharacterTextSplitter({
    chunkSize: 300,
    chunkOverlap: 30
});

var splitDocs;

splitter.splitDocuments(rawPdfDocuments).then((data) => {
    splitDocs = data;
    // console.log(splitDocs);
    console.log("Splitting done");
});

Promise { <pending> }

Splitting done


## Load documents in `Pinecone` vector store

In [10]:
pineconeStore.addDocuments(splitDocs).then((ids) => {
    console.log(ids);
    console.log("Successfully added documents to Pinecone!");
});

Promise { <pending> }

[
  '2000734b-2fe6-41c7-8bc0-52fcd2b92af0',
  '1403766e-de61-4000-b4c0-0fb8f7c60f62',
  '3d76ac39-6b6d-4009-a946-25c48bacbb25',
  'ebeaeece-5fb8-4b66-88a9-94e6ffce9deb',
  'c087cc58-2678-4e2b-84a8-b05f7c33018c',
  '3147dc42-da5f-46dd-988e-c3db01a07131',
  '6ff165e3-29e4-463f-9c7c-b4917102dbd5',
  '53229446-4fa2-412c-ae43-cd3d776080a8',
  '736f97bb-087d-429b-86cf-9a2b1a446f61',
  '0e6ae630-bccb-459e-adc0-df4670a0d425',
  '421dc057-05ac-4f89-b4ee-4b65b5170f79',
  'a55ff482-c5a9-4f7a-8aaf-45c35901b899',
  '163dfc53-c8cd-402d-becf-ff2a7c5c6fad',
  '85f2cfca-1c97-4fa5-ac1d-2dae4c5a3650',
  '702e9a50-204c-40ee-8c6a-9eb346c3fa28',
  '8bc4aac3-7c26-4cbf-9707-0017f761bbfd',
  '209e66b8-172b-465a-8b6f-f5009650d51b',
  '26f57249-d38d-4ef4-a531-e29e6aec97bc',
  'cd955053-8c3b-4c93-9c55-ed5a5821455d',
  'dadc18c5-11ee-42dd-8660-715eb3e1c731',
  '982a45bf-9fa7-4507-8d53-33c414f6c7e4',
  '0a19e0ce-bb86-4328-8f92-0740a7b0dbff',
  '2ea763ea-8f70-455c-959c-27f09f6eb4f2',
  '9473ac64-d74a-44e8-ae7b-a6e17

## Leverage Pinecone vector data

- now that we have loaded AWS Titan Embeddings into Pinecone vector store let's create a fresh chat client with a retriver using the loaded data
- no more loading of documents, only retrieving

In [11]:
require('dotenv').config({path: require('path').resolve(__dirname, '../../.env')});

var Bedrock = require('@langchain/community/llms/bedrock').Bedrock;
var createRetrievalChain = require('langchain/chains/retrieval').createRetrievalChain;

var model = new Bedrock({
    model_id:'amazon.titan-text-express-v1',
    temperature: 1,
    maxTokenCount: 512,
    topP: 0.9,
    verbose: true
});

var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;
var prompt = ChatPromptTemplate.fromTemplate(`
    Answer the user question.
    Context: {context}
    Question: {input}
`);

var Pinecone = require('@pinecone-database/pinecone').Pinecone;
var PineconeStore = require('@langchain/pinecone').PineconeStore;
var BedrockEmbeddings = require('@langchain/community/embeddings/bedrock').BedrockEmbeddings;

var embeddingsClient = new BedrockEmbeddings({
    model:'amazon.titan-embed-text-v2:0',
    region:'us-east-1'
});

var pineconeClient = new Pinecone({
    apiKey: process.env.PINECONE_API_KEY,
});

var pineconeIndex = pineconeClient.index(process.env.PINECONE_INDEX);
var pineconeStore = new PineconeStore(embeddingsClient, {
    pineconeIndex
});

var retriever = pineconeStore.asRetriever({k: 3});

var createStuffDocumentsChain = require("langchain/chains/combine_documents").createStuffDocumentsChain;
var combineDocsChain;

createStuffDocumentsChain({
    llm: model,
    prompt,
}).then((chain) => combineDocsChain = chain);

Promise { <pending> }

In [12]:
var retrievalChain;

createRetrievalChain({
    combineDocsChain,
    retriever
}).then((chain) => retrievalChain = chain);

Promise { <pending> }

In [13]:
var input = "Who is Andy Jassy?";

In [14]:
retrievalChain.invoke({
    input,
}).then((response) => console.log(response));

Promise { <pending> }

[llm/start] [1:llm:retrieval_chain] Entering LLM run with input: {
  "prompts": [
    "Human: \n    Answer the user question.\n    Context: I strongly believe that our best days are in front of us, and I look forward to working with my teammates at\nAmazon to make it so.\nSincerely,\nAndy Jassy\nPresident and Chief Executive Officer\nAmazon.com, Inc.\n\nI strongly believe that our best days are in front of us, and I look forward to working with my teammates at\nAmazon to make it so.\nSincerely,\nAndy Jassy\nPresident and Chief Executive Officer\nAmazon.com, Inc.\n\nI strongly believe that our best days are in front of us, and I look forward to working with my teammates at\nAmazon to make it so.\nSincerely,\nAndy Jassy\nPresident and Chief Executive Officer\nAmazon.com, Inc.\n    Question: Who is Andy Jassy?"
  ]
}
[llm/end] [1:llm:retrieval_chain] [2.06s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "\nAndy Jassy is the President and Chief Executive O